In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
params = {
    "data_dir" : "../data/myrealty_apartments.csv",
    "images_dir" : '../images/myrealty',
    "img_input_size" : 128,
    "batch_size" : 32,
    "shuffle" : True
}

In [11]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from ApartmentsDatasetPyTorch import ApartmentsDatasetPyTorch

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.Resize((params["img_input_size"], params["img_input_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Create the dataset
dataset = ApartmentsDatasetPyTorch(
    data_dir = params["data_dir"],
    images_dir = params["images_dir"], 
    transform = transform
)

# Create the dataloader
dataloader = DataLoader(
    dataset,
    batch_size = params["batch_size"],
    shuffle = params["shuffle"]
)

In [12]:
import torch
import torch.nn as nn
import timm

# Load the pretrained model
model = timm.create_model('inception_v4', pretrained=True)

# Freeze all the layers
for param in model.parameters():
    param.requires_grad = False

new_classifier = nn.Sequential(
    model.last_linear,
    nn.Linear(1000, 512), 
    nn.ReLU(),
    nn.Linear(512, 1)
)

# Replace the existing classifier in the model
model.last_linear = new_classifier

# Model training setup
model.train()  # Switch to training mode
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Example optimizer
loss_fn = torch.nn.MSELoss()  # Mean Squared Error Loss for regression

In [ ]:
num_epochs = 10
# Define loss function and optimizer

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, prices = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, prices.view(-1, 1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print("Running loss", running_loss)
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')

Running loss 77638590464.0
Running loss 183537672192.0
Running loss 286878244864.0
Running loss 376360951808.0
Running loss 437717114880.0
Running loss 493274865664.0
Running loss 576043671552.0
Running loss 643890479104.0
Running loss 761104064512.0
Running loss 814176972800.0
Running loss 882692370432.0
Running loss 915114801152.0
Running loss 993028814848.0
Running loss 1108766963712.0
Running loss 1191024285696.0
Running loss 1233064708096.0
Running loss 1263161223168.0
Running loss 1322509662208.0
Running loss 1375635171328.0
Running loss 1438308513792.0
Running loss 1488880588800.0
Running loss 1541074647040.0
Running loss 1606322272256.0
Epoch 1, Loss: 69840098793.73914
Running loss 25482844160.0
Running loss 45880471552.0
Running loss 80556224512.0
Running loss 132124053504.0
Running loss 172521992192.0
Running loss 220942479360.0
Running loss 280922075136.0
Running loss 342895861760.0
Running loss 386859941888.0
Running loss 466572296192.0
Running loss 495193931776.0
Running l